# Evaluation of CNN classification performance

In [55]:
muonic = False
hadronic = not muonic

In [56]:
import numpy as np

In [57]:
import tensorflow as tf

In [58]:
from sklearn.preprocessing import LabelEncoder

In [59]:
le = LabelEncoder()

In [60]:
from preprocessing import reshape_data

In [61]:
import uproot

In [62]:
import matplotlib.pyplot as plt

# plt.style.use(["science", "notebook"])

In [63]:
plt.rcParams["font.size"] = 14
plt.rcParams["axes.formatter.limits"] = -5, 4
plt.rcParams["figure.figsize"] = 6, 4
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

In [64]:
filename_test = "df_nu_CC_muonic_equal_test.root:df"

In [65]:
filename_test = "df_nu_CC_non-muonic_equal_new_test.root:df"

In [66]:
events_test = uproot.open(filename_test)

In [67]:
target = "nu_flavour"

target_pretty = "flavour"
target_LaTeX = "flavour"

In [68]:
events_test[target].num_entries

6791

In [69]:
y = np.abs(events_test[target])

In [70]:
le.fit(y)

LabelEncoder()

In [71]:
le.classes_

array([12, 16])

In [72]:
def event_generator():
    events = events_test
    for batch, report in events.iterate(step_size=1, report=True, library="np"):
        ys = le.transform(np.abs(batch[target]))
        for i in range(batch["X"].shape[0]):
            yield (
                batch["X"].astype(np.float16)[i],
                batch["X_mufilter"].astype(np.float16)[i],
                ys[i],
            )

In [73]:
gen = event_generator()

In [74]:
sample = gen.__next__()

In [75]:
generator_spec_0 = tf.type_spec_from_value(gen.__next__()[0])
generator_spec_1 = tf.type_spec_from_value(gen.__next__()[1])
generator_spec_2 = tf.type_spec_from_value(gen.__next__()[2])

In [76]:
print(generator_spec_0)

TensorSpec(shape=(3279, 116), dtype=tf.float16, name=None)


In [77]:
print(generator_spec_1)

TensorSpec(shape=(3279, 68), dtype=tf.float16, name=None)


In [78]:
print(generator_spec_2)

TensorSpec(shape=(), dtype=tf.int64, name=None)


In [79]:
ds_test = (
    tf.data.Dataset.from_generator(
        event_generator,
        output_signature=(
            generator_spec_0,
            generator_spec_1,
            generator_spec_2,
        ),
    )
    .map(reshape_data)
    .apply(tf.data.experimental.assert_cardinality(events_test.num_entries))
)

In [80]:
y_test = le.transform(np.abs(events_test[target]))

In [81]:
np.count_nonzero(y_test)

3069

In [82]:
batched_ds_test = ds_test.batch(30)

In [83]:
from tensorflow.keras.models import load_model

In [84]:
import tensorflow.keras.backend as K

K.set_image_data_format("channels_last")

In [85]:
model = load_model("CNN_jannu_logit_focus_nu_flavour_muvtau_n7465_e200.keras")

KeyboardInterrupt: 

In [ ]:
model = load_model("CNN_jannu_logit_focus_nu_flavour_evtau_n26824_e100.keras")

In [ ]:
model_name = model.name

In [ ]:
# test=retoy_model.predict(x=[x_test['scifi_h'], x_test['scifi_v'], x_test['us'], x_test['ds']])
y_pred = model.predict(batched_ds_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
from config import nu_mu_yield, nu_tau_yield, nu_e_yield, frac_muon, frac_hadr

In [ ]:
scaling_muon = np.array([nu_mu_yield, nu_tau_yield * frac_muon]).reshape(-1, 1)

In [ ]:
scaling_hadr = np.array([nu_e_yield, nu_tau_yield * frac_hadr]).reshape(-1, 1)

In [ ]:
def metric(threshold=0.5):
    TN, FP, FN, TP = confusion_matrix(y_test, (y_pred >= threshold).astype(int)).ravel()
    signal = TP / (TP + FN) * nu_tau_yield * (frac_muon if muonic else frac_hadr)
    background = FP / (FP + TN) * (nu_mu_yield if muonic else nu_e_yield)
    return -signal / np.sqrt(signal + background)

In [ ]:
from scipy.optimize import basinhopping

In [ ]:
res = basinhopping(metric, 0.2)

In [ ]:
optimum_threshold = res.x[0]

In [ ]:
# optimum_threshold = 0.5

In [ ]:
y_pred_bool = (y_pred >= optimum_threshold).astype(int)

In [ ]:
# plt.hist(y_pred_bool)
plt.hist(y_pred_bool)
plt.hist(y_test)

In [ ]:
print(classification_report(y_test, y_pred_bool))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
from plotting import watermark

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
# Plot non-normalized confusion matrix
titles_options = [
    ("Confusion matrix, without normalisation", None),
]
for title, normalize in titles_options:
    scale = 1.0
    if normalize == "scaled":
        normalize = "true"
        scale = scaling_hadr if hadronic else scaling_muon
    disp = ConfusionMatrixDisplay(
        confusion_matrix(
            le.inverse_transform(y_test.ravel()),
            le.inverse_transform(y_pred_bool.ravel()),
            normalize=normalize,
        )
        * scale,
        display_labels=[r"$\nu_e$" if hadronic else r"$\nu_\mu$", r"$\nu_\tau$"],
    )
    disp.plot(cmap=plt.cm.Blues, ax=ax)

    # disp.plot(ax=ax)
    # disp.ax_.set_title(title)

    # print(title)
    # print(disp.confusion_matrix)

watermark()

plt.savefig(f"plots/confusion_matrix_balanced_{model_name}.png")
plt.savefig(f"plots/confusion_matrix_balanced_{model_name}.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
# Plot non-normalized confusion matrix
titles_options = [
    ("Normalised confusion matrix", "true"),
]
for title, normalize in titles_options:
    scale = 1.0
    if normalize == "scaled":
        normalize = "true"
        scale = scaling_hadr if hadronic else scaling_muon
    disp = ConfusionMatrixDisplay(
        confusion_matrix(
            le.inverse_transform(y_test.ravel()),
            le.inverse_transform(y_pred_bool.ravel()),
            normalize=normalize,
        )
        * scale,
        display_labels=[r"$\nu_e$" if hadronic else r"$\nu_\mu$", r"$\nu_\tau$"],
    )
    disp.plot(cmap=plt.cm.Blues, ax=ax)

    # disp.plot(ax=ax)
    # disp.ax_.set_title(title)

    # print(title)
    # print(disp.confusion_matrix)

watermark()

plt.savefig(f"plots/confusion_matrix_normalised_{model_name}.png")
plt.savefig(f"plots/confusion_matrix_normalised_{model_name}.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
# Plot non-normalized confusion matrix
titles_options = [
    # ("Confusion matrix, without normalisation", None),
    # ("Normalised confusion matrix", "true"),
    ("Scaled confusion matrix", "scaled"),
]
for title, normalize in titles_options:
    scale = 1.0
    if normalize == "scaled":
        normalize = "true"
        scale = scaling_hadr if hadronic else scaling_muon
    disp = ConfusionMatrixDisplay(
        confusion_matrix(
            le.inverse_transform(y_test.ravel()),
            le.inverse_transform(y_pred_bool.ravel()),
            normalize=normalize,
        )
        * scale,
        display_labels=[r"$\nu_e$" if hadronic else r"$\nu_\mu$", r"$\nu_\tau$"],
    )
    disp.plot(cmap=plt.cm.Blues, ax=ax)

    # disp.plot(ax=ax)
    # disp.ax_.set_title(title)

    # print(title)
    # print(disp.confusion_matrix)

watermark()

plt.savefig(f"plots/confusion_matrix_scaled_{model_name}.png")
plt.savefig(f"plots/confusion_matrix_scaled_{model_name}.pdf")

In [ ]:
from sklearn.metrics import RocCurveDisplay

In [ ]:
RocCurveDisplay.from_predictions(y_test, y_pred)